In [ ]:
#https://mubaris.com/posts/movie-analysis/

# Imports

In [18]:
import pysrt



# Handling srt Subtitles

In [ ]:
# Loading the Subtitle
subs = pysrt.open('/home/mscuser/multimodal/multimodal_audio/Joey Bada$$ Talks Style, Activism, and Crystals-iY6dw9dEqfo.en.srt')
subtitles={}
len_subs=len(subs)
for i in range(len_subs):
    sub = subs[i]

    # Subtitle text
    text = sub.text
    text_without_tags = sub.text_without_tags

    # Start and End time
    start = sub.start.to_time()
    end = sub.end.to_time()
    subtitles[i]=[[text,text_without_tags,start,end]]
# Removing line and saving
#del subs[index]
#subs.save('/home/mscuser/multimodal/multimodal_audio/Joey Bada$$ Talks Style, Activism, and Crystals-iY6dw9dEqfo.en.srt')

In [19]:
print len(subtitles)

172


In [20]:
print subtitles[0]

[[u"she's like oh<c.colorCCCCCC> my god a rap video\n </c>", u"she's like oh my god a rap video\n ", datetime.time(0, 0, 11, 230000), datetime.time(0, 0, 11, 240000)]]
